In [1]:
class buyer():
    state = 0
    fund = 2000000
    stock = 0
    total = fund + stock
    fnum = '0'
    def __init__(self,fnum):
        self.fnum=fnum
    def buystock(self,price):
        self.stock=(self.fund/1.003)/price
        self.fund=0
        self.total=self.fund+(self.stock*price)
        self.state=1
        print("buy")
    def selstock(self,price):   
        self.fund=self.stock*price*0.997
        self.stock=0
        self.total=self.fund+(self.stock*price)
        self.state=0
        print("sell")
    def holstock(self,price):
        self.total=self.fund+(self.stock*price)
        print("hold")
    def make_choice(self,inp,price):
        if (self.state==0 and inp==1):
            self.buystock(price)
        elif(self.state==1 and inp==-1):
            self.selstock(price)
        else :
            self.holstock(price)      

In [6]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import talib
import mpl_finance as mpf
%matplotlib inline
import seaborn as sns
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import csv
import time
plt.rcParams['font.family']='SimHei' # 顯示中文('SimHei' for MacOS)
from sklearn import preprocessing

df1=pd.read_json('Tw50.json')
df1.sort_index(inplace=True)
df1=df1.fillna(0)
df1=df1.drop(range(14))
df1=df1.reset_index(drop=True)

List_Low=np.array(df1['最低價'])
List_Hig=np.array(df1['最高價'])
List_price=np.array(df1['收盤價'])
List_ans=np.array(0)
for i in range(19,List_price.size-1):
    LL=float(List_Low[i-1])
    LH=float(List_Hig[i-1])
    CL=float(List_Low[i])
    CH=float(List_Hig[i])
    RL=float(List_Low[i+1])
    RH=float(List_Hig[i+1])
    LP=float(List_price[i-1])
    CP=float(List_price[i])
    RP=float(List_price[i+1])
    cnt=2
    while(CP==LP):
        LP=float(List_price[i-cnt])
        cnt=cnt+1
    cnt=2
    while(CP==RP):
        RP=float(List_price[i+cnt])
        cnt=cnt+1
    if LP>CP and RP>CP:
        List_ans=np.append(List_ans,1)
    elif CP>LP and CP>RP:
        List_ans=np.append(List_ans,-1)
    else:
        List_ans=np.append(List_ans,0)
    '''
    if LL>CH and RL>CH :
        List_ans=np.append(List_ans,1)
    elif CL>=LH and CL>RH:
        List_ans=np.append(List_ans,-1)
    else:
        List_ans=np.append(List_ans,0)
    '''
List_ans=np.append(List_ans,0)
import sys
np.set_printoptions(threshold=sys.maxsize)
print(List_ans)

[ 0  1 -1  0  0  1  0  0 -1 -1  0  0  1  0  0 -1  0  0  1  0  0  0 -1  0
  1  0  0  0 -1  1  0 -1  1  0 -1  0  1  0 -1  1  0 -1  0  0  0  0  1 -1
  0  1  1 -1  0  1 -1  0  1 -1  0  0  1 -1  1  0  0  0 -1  0  1 -1  0  0
  0  1  0  0  0  0 -1  1 -1  0  1 -1  1 -1  0  0  1  0  0 -1  1  0 -1  0
  1  0 -1  0  1 -1  1  0  0 -1  1  0 -1  1  0 -1  0  1  0 -1  0  0  1  0
 -1  0  1  0  0 -1  0  0  1  0 -1  1  0  0  0 -1  0  1  1  0  0  0 -1  1
 -1  1 -1 -1  1 -1  1 -1  1 -1 -1  1  0 -1  0  0  1  0 -1  0  0  1  0  0
 -1  1 -1  1 -1  1 -1 -1  1  0 -1  1 -1  0  0  0  1 -1  1  0  0 -1  0  1
 -1  1  0  0  0  0  0 -1  1  1 -1  1 -1  1  0  0 -1  1  0  0 -1  1 -1  1
  0  0  0 -1  0  0  0  1  0  0 -1  0  0  1  0 -1  0  0  1  0  0  0 -1  0
  0  0  1  0  0 -1  1 -1 -1  0  1  1  0 -1  1  0 -1  1 -1  0  0  0  0  0
  1  1  0  0  0  0  0 -1  1  0 -1  1  0  0 -1  0  0  1 -1  0  0  0  1  0
 -1  0  0  1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1  0  0  1 -1  1 -1  1 -1  1
 -1  0  1  0 -1  1 -1 -1  0  0  0  1 -1  0  0  0  1

In [ ]:
Tom=buyer('0050')
for i in range(List_ans.size):
    Tom.make_choice(List_ans[i],float(List_price[i+18]))
print(Tom.total)

In [9]:
John=buyer('2727')
for i in range(List_ans.size):
    out=train(xxx.jpg)
    Tom.make_choice(out,float(List_price[i+18]))
print(John.total)

hold
buy
sell
hold
hold
buy
hold
hold
sell
hold
hold
hold
buy
hold
hold
sell
hold
hold
buy
hold
hold
hold
sell
hold
buy
hold
hold
hold
sell
buy
hold
sell
buy
hold
sell
hold
buy
hold
sell
buy
hold
sell
hold
hold
hold
hold
buy
sell
hold
buy
hold
sell
hold
buy
sell
hold
buy
sell
hold
hold
buy
sell
buy
hold
hold
hold
sell
hold
buy
sell
hold
hold
hold
buy
hold
hold
hold
hold
sell
buy
sell
hold
buy
sell
buy
sell
hold
hold
buy
hold
hold
sell
buy
hold
sell
hold
buy
hold
sell
hold
buy
sell
buy
hold
hold
sell
buy
hold
sell
buy
hold
sell
hold
buy
hold
sell
hold
hold
buy
hold
sell
hold
buy
hold
hold
sell
hold
hold
buy
hold
sell
buy
hold
hold
hold
sell
hold
buy
hold
hold
hold
hold
sell
buy
sell
buy
sell
hold
buy
sell
buy
sell
buy
sell
hold
buy
hold
sell
hold
hold
buy
hold
sell
hold
hold
buy
hold
hold
sell
buy
sell
buy
sell
buy
sell
hold
buy
hold
sell
buy
sell
hold
hold
hold
buy
sell
buy
hold
hold
sell
hold
buy
sell
buy
hold
hold
hold
hold
hold
sell
buy
hold
sell
buy
sell
buy
hold
hold
sell
buy
hold